# Bachelor 2021

In [55]:
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.stats import norm
import scipy

In [178]:
S0 = 100
K = 100
T = 1
r = 0.07
sigma = 0.2
n_simulations = 1000
n_steps = 252

In [179]:
def Geo_Mean(iterable):
    temp = np.array(iterable)
    return temp.prod()**(1.0/len(temp))

def Geo_Mean_Overflow(iterable):
    temp = np.log(iterable)
    return np.exp(temp.mean())

In [267]:
def Price_Asian_Option(S0, K, T, r, sigma, n_simulations, n_steps, Z):
    # Udvikling i prisen på det underliggende aktiv
    dt = T/n_steps
    Z = np.random.normal(0,1,size=(n_simulations, n_steps))
    S = np.zeros((n_simulations, n_steps))
    S[:,0] += S0
    for i in range(1, n_steps):
        S[:,i] += S[:,i-1]*np.exp((r-(sigma**2)/2)*dt+sigma*np.sqrt(dt)*Z[:,i])
    
    ### Formel kommer fra Paul Glasserman, 2004, s. 94
    ### Her skal evt. udvides med flere/andre processer ift. Geometrisk Brownian Motions

    # Gennemsnit af prisen på det underliggende aktiv per simulering
    avg = []
    for i in range(len(S)):
        avg.append(Geo_Mean_Overflow(S[i,:]))
    
    # Beregning af payoff for call
    c = []
    for i in range(len(avg)):
        c.append(avg[i] - K)
        c[i] = np.maximum(c[i], 0)

    # Beregning af payoff for put
    p = []
    for i in range(len(avg)):
        p.append(K - avg[i])
        p[i] = np.maximum(p[i], 0)

    # Gennemsnitligt payoff tilbagediskonteret
    payoff_call = np.mean(c)*np.exp(-r*T)
    payoff_put = np.mean(p)*np.exp(-r*T)
    
    return payoff_call, payoff_put


In [268]:
Price_Asian_Option(S0, K, T, r, sigma, n_simulations, n_steps)

TypeError: Price_Asian_Option() missing 1 required positional argument: 'Z'

In [264]:
# Udvikling i prisen på det underliggende aktiv
dt = T/n_steps
Z = np.random.normal(0,1,size=(n_simulations, n_steps))
S = np.zeros((n_simulations, n_steps))
S[:,0] += S0
for i in range(1, n_steps):
    S[:,i] += S[:,i-1]*np.exp((r-(sigma**2)/2)*dt+sigma*np.sqrt(dt)*Z[:,i])

### Formel kommer fra Paul Glasserman, 2004, s. 94
### Her skal evt. udvides med flere/andre processer ift. Geometrisk Brownian Motions


# Gennemsnit af prisen på det underliggende aktiv per simulering
avg = []
for i in range(len(S)):
    avg.append(Geo_Mean_Overflow(S[i,:]))
    
# Beregning af payoff for call
c = []
for i in range(len(avg)):
    c.append(avg[i] - K)
    c[i] = np.maximum(c[i], 0)

# Beregning af payoff for put
p = []
for i in range(len(avg)):
    p.append(K - avg[i])
    p[i] = np.maximum(p[i], 0)

# Gennemsnitligt payoff
payoff_call = np.mean(c)
payoff_put = np.mean(p)

# Tilbagediskontering
call = payoff_call*np.exp(-r*T)
put = payoff_put*np.exp(-r*T)

In [265]:
call

6.028153201521012

In [266]:
put

2.9212784580529614

**Tjek med Black-Scholes**

$C_G = S_0 e^{(b-r)T} \Phi(d_1) - K e^{-rT}\Phi(d_2)$

$P_G = K e^{-rT}\Phi(-d_2) - S_0 e^{(b-r)T} \Phi(-d_1)$

$\sigma_G = \frac{\sigma}{\sqrt{3}}$

$b = \frac{1}{2}(r-\frac{1}{2} \sigma_G^2)$

$d_1 = \frac{log(\frac{S_0}{K})+(b+\frac{1}{2}\sigma_G^2)T}{\sigma_G\sqrt{T}}$

$d_2 = d_1 - \sigma_G \sqrt{T}$

In [190]:
sigma_G = sigma/np.sqrt(3)

b = (1/2)*(r-(1/2)*(sigma_G**2))

d1 = (np.log(S0/K)+(b+1/2*(sigma_G**2))*T)/(sigma_G*np.sqrt(T))

d2 = d1 - sigma_G*np.sqrt(T)

C_G = S0*np.exp((b-r)*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

P_G = K*np.exp(-r*T)*norm.cdf(-d2) - S0*np.exp((b-r)*T)*norm.cdf(-d1)

In [191]:
C_G

6.024544283554853

In [192]:
P_G

3.024717268581746